# GINI METRICS(Especially used in industry for Frequency Severity modelling)

In [ ]:
predictions = [0.9, 0.3, 0.8, 0.75, 0.65, 0.6, 0.78, 0.7, 0.05, 0.4, 0.4, 0.05, 0.5, 0.1, 0.1]
actual = [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
import numpy as np

In [ ]:
all = np.asarray(np.c_[actual, predictions, np.arange(len(actual))], dtype=np.float)
all # arranging in an array

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


array([[ 1.  ,  0.9 ,  0.  ],
       [ 1.  ,  0.3 ,  1.  ],
       [ 1.  ,  0.8 ,  2.  ],
       [ 1.  ,  0.75,  3.  ],
       [ 1.  ,  0.65,  4.  ],
       [ 1.  ,  0.6 ,  5.  ],
       [ 0.  ,  0.78,  6.  ],
       [ 0.  ,  0.7 ,  7.  ],
       [ 0.  ,  0.05,  8.  ],
       [ 0.  ,  0.4 ,  9.  ],
       [ 0.  ,  0.4 , 10.  ],
       [ 0.  ,  0.05, 11.  ],
       [ 0.  ,  0.5 , 12.  ],
       [ 0.  ,  0.1 , 13.  ],
       [ 0.  ,  0.1 , 14.  ]])

In [ ]:
all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]
all# sort based on predictions from large to small

array([[ 1.  ,  0.9 ,  0.  ],
       [ 1.  ,  0.8 ,  2.  ],
       [ 0.  ,  0.78,  6.  ],
       [ 1.  ,  0.75,  3.  ],
       [ 0.  ,  0.7 ,  7.  ],
       [ 1.  ,  0.65,  4.  ],
       [ 1.  ,  0.6 ,  5.  ],
       [ 0.  ,  0.5 , 12.  ],
       [ 0.  ,  0.4 ,  9.  ],
       [ 0.  ,  0.4 , 10.  ],
       [ 1.  ,  0.3 ,  1.  ],
       [ 0.  ,  0.1 , 13.  ],
       [ 0.  ,  0.1 , 14.  ],
       [ 0.  ,  0.05,  8.  ],
       [ 0.  ,  0.05, 11.  ]])

In [ ]:
totalLosses = all[:, 0].sum()
totalLosses# sum o f all the `actual` column

6.0

In [ ]:
giniSum = all[:, 0].cumsum().sum() / totalLosses# for `actual column` finding the total of cumulative sum of the actual column array/sum of all of the actual column
giniSum

10.833333333333334

In [ ]:
giniSum -= (len(actual) + 1) / 2.
giniSum

2.833333333333334

In [ ]:
giniSum / len(actual)# gini coefficient

0.18888888888888894

*ALL code put together*

In [ ]:
assert (len(actual) == len(predictions))
all = np.asarray(np.c_[actual, predictions, np.arange(len(actual))], dtype=np.float)# arranging in an array
all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]# sort based on predictions from large to small
totalLosses = all[:, 0].sum()# sum of all the actual column
giniSum = all[:, 0].cumsum().sum() / totalLosses# for actual column finding the total of cumulative sum of the actual column array/sum of all of the actual column
giniSum -= (len(actual) + 1) / 2.
giniSum / len(actual)# gini coefficient

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


0.18888888888888894

# a different way of coding the same metric is:


In [ ]:
def mygini(sortby, pred, actual, weight = None, n=10, plot = True, std = None):
  if weight is None:
    weight = np.ones_like(pred)

  def weighted_std(values, weight):
    m = np.average(values, weights = weight)
    return np.sqrt(np.average((values - m)**2, weights = weight))
  r = np.vstack((sortby, pred, actual, weight)).T
  r = r[sortby.argsort()].T
  cumm_y = np.cumsum(r[2]*r[3])
  total_w = np.sum(weight)
  gini = 1-2*(np.sum(cumm_y*r[3])/(np.sum(r[2]*r[3])*total_w))
  return gini

# calling the "mygini" function
fit_model = poisson_model.fit()
predictions = fit_model.predict(X_train)
mygini(predictions, predictions, y_train)

In [ ]:
from sklearn.metrics import auc
def lorenz_curve(y_true, y_pred, exposure):
    y_true, y_pred = np.asarray(y_true), np.asarray(y_pred)
    exposure = np.asarray(exposure)

    # order samples by increasing predicted risk:
    ranking = np.argsort(y_pred)
    ranked_exposure = exposure[ranking]
    ranked_pure_premium = y_true[ranking]
    cumulated_claim_amount = np.cumsum(ranked_pure_premium * ranked_exposure)
    cumulated_claim_amount /= cumulated_claim_amount[-1]
    cumulated_samples = np.linspace(0, 1, len(cumulated_claim_amount))
    return cumulated_samples, cumulated_claim_amount

ordered_samples, cum_claims = lorenz_curve(df_test["PurePremium"], y_pred, df_test["Exposure"])
gini = 1 - 2 * auc(ordered_samples, cum_claims)